In [11]:
# General libraries
import pandas as pd
import numpy as np
from collections import defaultdict
import random
from random import randrange

In [12]:
# dictionaries with the structure source -> array of followings
dictionary = defaultdict(list)
# simple datafram to store the sources and the count of source followings
data = pd.DataFrame([])

file = open('data/train.txt', 'r') 
lines = file.readlines() 
count = 0
for line in lines: 
    count = count + 1
    split_string = list(map(int,line.strip().split("\t")))
    dictionary[split_string[0]] = []
    if (len(split_string) > 1):
        dictionary[split_string[0]] = sorted(split_string[1:len(split_string)])
        
    data = data.append(pd.DataFrame({'Source': split_string[0], 'Source_Followings': len(dictionary[split_string[0]]) }, index=[0]), ignore_index=True)

In [13]:
def followingInCommon(node1,node2):  
    list1 = dictionary[node1]
    list2 = dictionary[node2]  
    common_elements = set(list1).intersection(list2) 
    return len(common_elements)
    
# Test    
followingInCommon(3849054,161276) 

1137

In [ ]:
def indirectFollowings(node1,node2):  
    list1 = dictionary[node1]
    list2 = dictionary[node2]  
    common_elements = set(list1).intersection(list2) 
    return len(common_elements)
    
# Test    
followingInCommon(3849054,161276) 

In [17]:
real_edges = pd.read_csv("data_generated/real_edges.csv", sep='\t')
real_edges2 = pd.read_csv("data_generated/real_edges2.csv", sep='\t')
fake_edges = pd.read_csv("data_generated/fake_edges3.csv", sep='\t')
#fake_edges2 = pd.read_csv("data_generated/fake_edges2.csv", sep='\t')

In [18]:
real_edges['Real'] = 1
real_edges2['Real'] = 1
fake_edges['Real'] = 0
#fake_edges2['Real'] = 0
frames = [real_edges, real_edges2,fake_edges]
dataset = pd.concat(frames, ignore_index=True)

In [19]:
dataset.shape

(7955, 6)

In [20]:
dataset['Common_Followings'] = dataset.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
dataset['Com_Followings_Ratio'] = dataset['Common_Followings']/dataset['Source_Followings']
dataset['Followers_Ratio'] = dataset['Sink_Followers']/dataset['Source_Followers']
# Saving real edges
dataset.to_csv("data_models/dataset2.csv", sep='\t', index=False)

In [4]:
test_data = pd.read_csv("data_generated/test_data.csv", sep='\t')

In [5]:
test_data.head()

,Source,Sink,Source_Followings,Source_Followers,Sink_Followers
0,3563811,3600160,21,3,29
1,2052043,1401960,71,13,9
2,4517994,1690636,205,80,17
3,1660006,4349447,506,32,36
4,581111,1882617,18,5,46


In [10]:
count = 0
for index, row in test_data.iterrows():
    
    if row['Sink'] in data.Source.values:
        count += 1
        
print(count)

376


In [23]:
test_data['Common_Followings'] = test_data.apply(lambda x: followingInCommon(x['Source'], x['Sink']), axis=1)
test_data['Com_Followings_Ratio'] = test_data['Common_Followings']/test_data['Source_Followings']
test_data['Followers_Ratio'] = test_data['Sink_Followers']/test_data['Source_Followers']
test_data.to_csv("data_models/test_data.csv", sep='\t', index=False)
